### MANDATORY ASSIGNMENT 2

In [138]:
from sklearn import datasets

In [139]:
import numpy as np

In [140]:
iris = datasets.load_iris()

In [141]:
X = iris.data
Y = iris.target

#### Task 1) data exploration

In [142]:
len(X)

150

In [143]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [144]:
print(np.min(X), np.max(X))
print(np.min(Y), np.max(Y))

0.1 7.9
0 2


##### 2) Encoding and pre-processing

In [145]:
#at first we normalize the data from 0 to pi, and then implement angle encoding
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from qiskit_algorithms.optimizers import GradientDescent
import random

In [146]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

In [147]:
def angle_encoding(qc, sample):
    for qubit in range(len(qc.qubits)):
        qc.rx(sample[qubit], qubit)

    

##### 3) choosing Loss function

In [148]:
from sklearn.metrics import accuracy_score


##### 4) splitting data

In [149]:
from sklearn.model_selection import train_test_split

In [150]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 15% validation, 15% testing

In [151]:
num_qubits = 4
num_layers = 3

In [152]:
int('1111', 2)

15

In [153]:
theta = '\u03B8'

def real_amplitudes(data_point,parameters, layers = num_layers):
    qc = QuantumCircuit(num_qubits)
    angle_encoding(qc, data_point)

    param_index = 0

    for layer in range(layers):
        for qubit in range(len(qc.qubits)):
            qc.ry(parameters[param_index], qubit)
            param_index += 1
        qc.barrier()
        
        for qubit in range(len(qc.qubits)-1):
            qc.cx(qubit, qubit+1)
        qc.barrier()

    return qc
    

In [154]:
def data_decoding(output):
    return int(output, 2) % 3 

In [155]:
def objective_function(updated_params):
    total_correct_class_probability = 0
    for x, y in zip(X_train, y_train):
        qc = real_amplitudes(x, updated_params)
        qc.measure_all()
        shots = 100
        backend = AerSimulator()
        tqc = transpile(qc, backend)
        job = backend.run(tqc, shots=shots)
        result = job.result()
        counts = result.get_counts(qc)

        total_correct_class_probability += sum(count for output, count in counts.items() if data_decoding(output) == y_train[0]) / shots

    cost = 1 - (total_correct_class_probability / len(X_train))

    print(f"Parameters: {updated_params} cost: {cost}")
    return cost
    

In [156]:
print(X_train[0], y_train[0])

[1.04719755 0.52359878 1.43767799 1.17809725] 1


In [157]:
initial_parameters = np.zeros(num_layers * num_qubits)

# Gradient Descent optimizer
optimizer = GradientDescent(maxiter=100, learning_rate=0.01)

# Optimize the parameters
optimized = optimizer.minimize(fun=objective_function, x0=initial_parameters)

print("Optimized Parameters:", optimized.x)
print("Minimum Cost:", optimized.fun)


Parameters: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] cost: 0.6395238095238095
Parameters: [0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ] cost: 0.6406666666666667
Parameters: [0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ] cost: 0.6412380952380952
Parameters: [0.   0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.  ] cost: 0.6461904761904762
Parameters: [0.   0.   0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.  ] cost: 0.6343809523809523
Parameters: [0.   0.   0.   0.   0.01 0.   0.   0.   0.   0.   0.   0.  ] cost: 0.6374285714285713
Parameters: [0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.   0.   0.  ] cost: 0.6360952380952382
Parameters: [0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.   0.  ] cost: 0.6393333333333332
Parameters: [0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.  ] cost: 0.6338095238095238
Parameters: [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.  ] cost: 0.6406666666666666
Parameters: [0.   0.   0.   0.   0

In [162]:
def predict(data_point, optimized_params):
    qc = real_amplitudes(data_point, optimized_params)
    qc.measure_all()

    backend = AerSimulator()
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=1000)
    result = job.result()
    counts = result.get_counts(qc)

    decoded_counts = {}
    for output, count in counts.items():
        decoded_output = data_decoding(output)
        if decoded_output in decoded_counts:
            decoded_counts[decoded_output] += count
        else:
            decoded_counts[decoded_output] = count
    
    prediction = max(decoded_counts, key=decoded_counts.get)
    
  

    return prediction

In [163]:
def predict_dataset(X, optimized_params):
    return [predict(data_point, optimized_params) for data_point in X]

In [164]:
predictions = predict_dataset(X_test, optimized.x)
predictions

[0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]

In [166]:
score = accuracy_score(y_test, predictions)
score

0.5652173913043478

In [167]:
y_test

array([0, 2, 2, 0, 2, 1, 1, 0, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 2, 0, 0,
       2])